# Load env and files

In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from fossil_classification import *
from enrich_holdings import *
from reports_etl import *
from holdings_analysis import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.3f}".format

In [4]:
response_path = "data/downloaded reports/company reports 2022Q1/"
all_holdings_cls = pd.read_csv(response_path+"all_holdings_cls_all_ids.csv", dtype=holdings_dtypes())
# all_holdings_cls = load_mappings_and_add_ids_to_holdings(all_holdings_cls)

/Users/urimarom/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,7,13,20,22,23,24,25,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Let the Analysis begin!

In [5]:
pd.DataFrame(all_holdings_cls.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
ביטוח,"3,547,094,875.89"
גמל,"5,204,786,509.43"
פנסיה,"4,950,848,935.83"


## Get Company level stats over time

### Filter results to include only major companies

In [5]:
all_holdings_cls_filtered = filter_major_companies(all_holdings_cls)
pd.DataFrame(all_holdings_cls_filtered.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
ביטוח,"3,385,873,064.68"
גמל,"4,451,699,217.37"
פנסיה,"4,534,354,836.33"


### * should include other subsidiary funds as well?

In [7]:
# included_comp_group = all_holdings_cls_filtered["ParentCorpGroup"].unique()
# included_comp = all_holdings_cls_filtered["ParentCorpName"].unique()
# all_comp = all_holdings_cls["ParentCorpName"].unique()
# not_included_comp = [c for c in all_comp if c not in included_comp]
# print([c for c in not_included_comp if any(g in re.split('-| ',c) for g in included_comp_group)])

# pd.DataFrame(
#     all_holdings_cls[
#     all_holdings_cls["ParentCorpName"].isin(not_included_comp)
#     ].groupby(["ParentCorpName", "SystemName"])["שווי"].sum()
# ).sort_values("שווי", ascending=False)

In [8]:
company_stats_since_2020 = get_summary(all_holdings_cls_filtered, 'ReportPeriodDate', 'ParentCorpGroup')
company_stats_since_2020 = company_stats_since_2020.sort_values(
    ["ReportPeriodDate", "ParentCorpGroup"], ascending=(False, True)
)

In [9]:
# company_stats_since_2020[company_stats_since_2020['ParentCorpGroup'] == 'מגדל']

In [10]:
company_stats_since_2020.to_csv(response_path+"company_stats_since_2020.csv", index=False)

In [11]:
company_system_stats_since_2020 = get_summary(
    all_holdings_cls_filtered,
    'ReportPeriodDate', 'ParentCorpGroup', 'SystemName'
)

In [12]:
company_system_stats_since_2020.to_csv(response_path+"company_system_stats_since_2020.csv", index=False)

In [13]:
company_system_holding_type_stats_since_2020 = get_summary(
    all_holdings_cls_filtered,
    'ReportPeriodDate', 'ParentCorpGroup', 'SystemName', 'holding_type'
)

In [14]:
company_system_holding_type_stats_since_2020.to_csv(
    response_path+"company_system_holding_type_stats_since_2020.csv", index=False)

### Export aggregated data for quarterly org ranking

In [15]:
midrag_agg = get_latest_q_ranking_agg_from_holdings(filter_major_companies(all_holdings_cls))

In [16]:
midrag_agg.to_csv(
    response_path+"midrag_agg_latest_q.csv", index=False
)

Upload results to G-Drive: https://drive.google.com/drive/u/2/folders/1n0F9JpyJ4LdWxLc4-a8XaRaMV20qo1Fx

## Mergers - update
### 1. update holdings data according to system mergers

In [6]:
all_holdings_majors = filter_major_companies(all_holdings_cls)
# including subsidiaries
# majors_after_mergers = filter_major_companies(all_holdings_cls, include_subsidiaries=True)

other_quarters = all_holdings_majors[
    all_holdings_majors["ReportPeriodDate"] != '2021-09-30'
#     majors_after_mergers["ReportPeriodDate"].max()
]

# leave only mergers quarter (2021 Q3)
majors_after_mergers = all_holdings_majors[
    all_holdings_majors["ReportPeriodDate"] == '2021-09-30'
#     majors_after_mergers["ReportPeriodDate"].max()
]

# change data according to mergers
majors_after_mergers = update_system_owners(majors_after_mergers, "גמל", "הלמן", "הפניקס")
majors_after_mergers = update_system_owners(majors_after_mergers, "פנסיה", "הלמן", "מיטב")
majors_after_mergers = update_system_owners(majors_after_mergers, "גמל", "פסגות", "אלטשולר")
majors_after_mergers = update_system_owners(majors_after_mergers, "פנסיה", "פסגות", "הראל")

moving 15,677,553.290000001 from הלמן גמל to הפניקס גמל
moving 0.0 from הלמן פנסיה to מיטב פנסיה
moving 49,780,952.0 from פסגות גמל to אלטשולר גמל
moving 3,964,347.4000000004 from פסגות פנסיה to הראל פנסיה


/Users/urimarom/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [7]:
# adding back to other periods
all_holdings_after_mergers = pd.concat([majors_after_mergers, other_quarters])
pd.DataFrame(all_holdings_after_mergers.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
ביטוח,"3,385,873,064.68"
גמל,"4,451,699,217.37"
פנסיה,"4,534,354,836.33"


In [19]:
company_stats_after_mergers = get_summary(
    all_holdings_after_mergers,
    'ReportPeriodDate', 'ParentCorpGroup'
)
# company_system_stats_after_mergers.sort_values("שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים")
# company_stats_after_mergers.sort_values("שיעור פוסילי במניות ואגח קונצרני סחירים")[[
#     "ReportPeriodDate",
#     "ParentCorpGroup",
#     "שווי",
#     "שווי במניות ואגח קונצרני סחירים",
#     "שיעור פוסילי במניות ואגח קונצרני סחירים"
# ]]

# company_system_stats_after_mergers["ratio"] = (
#     company_system_stats_after_mergers["שיעור פוסילי במניות ואגח קונצרני סחירים"] / 
#     company_system_stats_after_mergers["שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים"]
# )

company_stats_after_mergers.to_csv(response_path+"company_stats_since_2020_after_mergers.csv", index=False)

### Export aggregated data <u> after mergers </u> for quarterly org ranking

In [20]:
midrag_agg_after_mergers = get_latest_q_ranking_agg_from_holdings(all_holdings_after_mergers)

In [21]:
midrag_agg_after_mergers.to_csv(
    response_path+"midrag_after_mergers.csv", index=False
)

# Export holdings data for quarterly ranking

In [22]:
q = '2022 רבעון 1'
holding_types_for_ranking = ['מניות', 'אג"ח קונצרני']
cols = [
    'שם המנפיק/שם נייר ערך',
    'מספר ני"ע',
    'ISIN',
    'מספר מנפיק',
    'מספר תאגיד',
    'LEI',
    'ענף מסחר',
    'שווי',
    'שעור מנכסי אפיק ההשקעה',
    'שעור מסך נכסי השקעה',
    'שעור מערך נקוב מונפק',
    'holding_type',
    'SystemName',
    'ParentCorpGroup',
    'is_fossil',
    'שווי פוסילי'
]
holdings_for_ranking = filter_major_companies(all_holdings_cls)
holdings_for_ranking = holdings_for_ranking.loc[
    (holdings_for_ranking['ReportPeriodDesc'] == q) &
    (holdings_for_ranking["holding_type"].isin(holding_types_for_ranking)),
    cols
]
# use ISIN when Israeli sec num is not available
holdings_for_ranking['מספר ני"ע'] = holdings_for_ranking['מספר ני"ע'].fillna(holdings_for_ranking['ISIN'])
# use Israeli corp num and then LEI when issuer number is not available
holdings_for_ranking['מספר מנפיק'] = holdings_for_ranking['מספר מנפיק'].fillna(holdings_for_ranking['מספר תאגיד'])
holdings_for_ranking['מספר מנפיק'] = holdings_for_ranking['מספר מנפיק'].fillna(holdings_for_ranking['LEI'])
holdings_for_ranking.drop(['ISIN', 'מספר תאגיד', 'LEI'], axis=1, inplace=True)
holdings_for_ranking.rename({
    "holding_type": "סוג החזקה",
    "SystemName":  "אפיק",
    "ParentCorpGroup": "גוף"
}, axis=1, inplace=True)

# holdings_for_ranking[
#     (holdings_for_ranking["is_fossil"] == 1) &
#     ~(holdings_for_ranking["שווי פוסילי"] > 0)
# ]

holdings_for_ranking.to_csv(response_path+"quarterly_ranking_data.csv", index=False)

## Export csv per institution

In [23]:
inst_cols = [c for c in holdings_for_ranking.columns if (c != 'גוף') & ~(c.startswith("שעור"))]
for inst in get_major_institutions_list():
    holding_per_inst = holdings_for_ranking.loc[
        (holdings_for_ranking["גוף"] == inst),
        inst_cols
    ].sort_values("שווי פוסילי", ascending=False)
    holding_per_inst.to_csv(response_path+inst+".csv", index=False)

# Get a list of all fossil holdings for major institutions for a given quarter

In [24]:
q = ['2022 רבעון 1']
inst = get_major_institutions_list()
fossil_holdings_last_q = group_fossil_holdings_quarters_institutions(
    all_holdings_cls_filtered,
    quarters=q,
    institutions = inst
)

fossil_holdings_last_q.to_csv("analysis/last_q_major_insts_fossil_holdings.csv", index=False)

total Israeli fossil holdings: 52427764.35988916
total fossil holdings with il_sec_num: 52427764.359889165
total fossil holdings without il_sec_num: 12137408.36900178
12137408.36900178
after adding Israeli and non-Israeli: 64565172.72889094


# Compare Fossil Holdings over quarters
## 1. for specific companies

In [8]:
company_for_comparison = ['מגדל']
quarters_for_comparison = [
    '2021 רבעון 4',
    '2022 רבעון 1'
]
# comparison = compare_fossil_holdings_over_quarters(all_holdings_cls_filtered, quarters_for_comparison, company_for_comparison)
comparison = compare_fossil_holdings_over_quarters(all_holdings_after_mergers, quarters_for_comparison, company_for_comparison)
comparison.sort_values("name", ascending=False)

total Israeli fossil holdings: 17855903.791571625
total fossil holdings with il_sec_num: 17855903.791571625
total fossil holdings without il_sec_num: 3435697.5325038815
3435697.5325038815
after adding Israeli and non-Israeli: 21291601.32407551


,name,id,fossil_sum_prev_q,quantity_sum_prev_q,fossil_sum_curr_q,quantity_sum_curr_q,quantity_diff,quantity_diff_pct,fossil_sum_diff,fossil_sum_diff_pct
59,תמר פטרוליום,1689,"425,157.310","428,209,391.954","430,678.746","415,065,105.448","-13,144,286.505",-0.031,"5,521.435",0.013
58,שפיר,1633,"686,602.975","81,018,074.411","756,634.738","23,831,023.921","-57,187,050.490",-0.706,"70,031.764",0.102
57,שיכון ובינוי,1068,"344,093.417","17,265,098.713","331,489.729","17,265,090.046",-8.667,-0.000,"-12,603.688",-0.037
56,רציו יהש,394,"66,803.770","39,599,152.053","89,098.047","39,599,132.166",-19.887,-0.000,"22,294.278",0.334
55,רפק,769,11.333,335.000,10.445,335.000,0.000,0.000,-0.888,-0.078
54,רימון מועמדת,1928,"87,867.866","2,294,200.149","109,341.524","2,294,199.001",-1.148,-0.000,"21,473.659",0.244
53,קנון,1635,15.800,100.000,21.140,100.000,0.000,0.000,5.340,0.338
52,פז נפט,1363,"808,576.732","448,317,831.455","827,943.955","419,081,465.740","-29,236,365.715",-0.065,"19,367.223",0.024
51,סופרגז,1804,"76,952.342","1,221,853.628","78,867.745","1,249,686.983","27,833.355",0.023,"1,915.404",0.025
50,נתיבי גז,1418,"42,147.317","31,551,427.825","39,344.629","31,551,427.747",-0.078,-0.000,"-2,802.688",-0.066


In [28]:
company_quarter = company_for_comparison[0] + " - " + quarters_for_comparison[1]
comparison.to_csv("analysis/fossil holdings comparison - " + company_quarter + ".csv", index=False)

## add profit/loss per quarter
treating changes in quantity as if they took place at the last day of the quarter:
* If quantity went up (securities were bought): calculate the change for prev_q amount
* If quantity went down (securities were sold): calculate the change for prev_q amount 

previous quarter quantity * (current quarter price - previous quarter price)

In [11]:
comparison["price_diff"] = (
    (comparison["fossil_sum_curr_q"] / comparison["quantity_sum_curr_q"]) -
    (comparison["fossil_sum_prev_q"] / comparison["quantity_sum_prev_q"])
)

comparison["value_diff"] = comparison["price_diff"] * comparison["quantity_sum_prev_q"]

In [38]:
print(comparison["value_diff"].sum())
comparison

2707216.7233719407


,name,id,fossil_sum_prev_q,quantity_sum_prev_q,fossil_sum_curr_q,quantity_sum_curr_q,quantity_diff,quantity_diff_pct,fossil_sum_diff,fossil_sum_diff_pct,price_diff,value_diff
0,BARRICK GOLD,CA0679011084,6.438,110.000,0.000,0.000,-110.000,-1.000,-6.438,-1.000,nan,nan
1,BERKSHIRE HATHWAY,5493000C01ZX7D35SD85,270.552,290.000,206.235,184.000,-106.000,-0.366,-64.317,-0.238,0.188,54.492
2,BIP US EQUITY,549300WEH5R2OODI7Y90,16.470,88.000,0.000,0.000,-88.000,-1.000,-16.470,-1.000,nan,nan
3,BPLN,549300CRVT18MXX0AG93,"124,034.558","36,999,999.866","118,964.546","37,000,000.083",0.217,0.000,"-5,070.012",-0.041,-0.000,"-5,070.012"
4,CHEVRON,US1667641005,15.339,42.000,21.720,42.000,0.000,0.000,6.382,0.416,0.152,6.382
60,CIVITAS RESOURCE,IIVIHPH28S64K95M2381,0.000,0.000,37.928,200.000,200.000,inf,37.928,inf,nan,nan
5,CQP,5493005UEC8AZ34LDV29,"106,487.110","32,099,999.944","102,882.887","32,100,000.133",0.189,0.000,"-3,604.223",-0.034,-0.000,"-3,604.224"
6,DISCOVERY,US25470F1049,62.352,820.000,57.302,724.000,-96.000,-0.117,-5.051,-0.081,0.003,2.547
7,DLET US EQUITY,MTLVN9N7JE8MIBIJ1H73,8.882,350.000,12.439,350.000,0.000,0.000,3.557,0.400,0.010,3.557
8,ELECTRICITE DE FRANCE,549300X3UK4GG3FNMO06,"187,808.164","43,699,999.889","169,975.846","43,700,000.081",0.193,0.000,"-17,832.318",-0.095,-0.000,"-17,832.319"


In [14]:
# divide to מניות and אגח according to request (ugly but no time...)

In [41]:
def fossil_profit_per_q_by_holding_type(holdings, quarters, company, holding_types):
    results = []
    for i in range(len(quarters)-1):
        for h in holding_types:
            holdings_filtered_by_type = holdings[holdings["holding_type"] == h]
            quarters_for_comparison = [quarters[i], quarters[i+1]]
            comparison = compare_fossil_holdings_over_quarters(holdings_filtered_by_type, quarters_for_comparison, company)
            comparison["prev_price"] = comparison["fossil_sum_prev_q"] / comparison["quantity_sum_prev_q"]
            comparison["curr_price"] = comparison["fossil_sum_curr_q"] / comparison["quantity_sum_curr_q"]
            comparison["price_diff"] = comparison["curr_price"] - comparison["prev_price"]
            comparison["price_diff_pct"] = comparison["price_diff"] / comparison["prev_price"]
            comparison["value_diff"] = comparison["price_diff"] * comparison["quantity_sum_prev_q"]
            # look into rows with abnormal price diff (might mean something changed in number of shares?)
            print(comparison[
                (comparison["price_diff_pct"] > 0.5) |
                (comparison["price_diff_pct"] < -0.5)
            ])
            profit_end_q = comparison["value_diff"].sum()
            total_end_q = comparison["fossil_sum_curr_q"].sum()
            results.append({
                'quarter': quarters[i+1],
                'holding_type': h,
                'profit': profit_end_q,
                'total': total_end_q
            })
    return pd.DataFrame(results)
            

In [43]:
quarters = [
    '2020 רבעון 4',
    '2021 רבעון 1',
    '2021 רבעון 2',
    '2021 רבעון 3',
    '2021 רבעון 4',
    '2022 רבעון 1'
]

holding_types = ['מניות', 'אג"ח קונצרני']

fossil_profits_by_type_and_q = fossil_profit_per_q_by_holding_type(all_holdings_after_mergers, quarters, ['מגדל'], holding_types)
fossil_profits_by_type_and_q.to_clipboard(index=False)
fossil_profits_by_type_and_q

total Israeli fossil holdings: 8028832.210682096
total fossil holdings with il_sec_num: 8028832.210682097
total fossil holdings without il_sec_num: 644222.2578773967
644222.2578773967
after adding Israeli and non-Israeli: 8673054.468559492
            name    id  fossil_sum_prev_q  quantity_sum_prev_q  \
18  תמר פטרוליום  1689         18,130.720        6,621,884.761   

    fossil_sum_curr_q  quantity_sum_curr_q  quantity_diff  quantity_diff_pct  \
18         28,816.053        6,618,294.323     -3,590.438             -0.001   

    fossil_sum_diff  fossil_sum_diff_pct  prev_price  curr_price  price_diff  \
18       10,685.333                0.589       0.003       0.004       0.002   

    price_diff_pct  value_diff  
18           0.590  10,700.966  
total Israeli fossil holdings: 5726685.117467999
total fossil holdings with il_sec_num: 5726685.117467999
total fossil holdings without il_sec_num: 4055697.1411084626
4055697.1411084626
after adding Israeli and non-Israeli: 9782382.2585764

,quarter,holding_type,profit,total
0,2021 רבעון 1,מניות,"360,559.152","4,513,090.385"
1,2021 רבעון 1,"אג""ח קונצרני","86,010.292","4,673,533.462"
2,2021 רבעון 2,מניות,"289,843.676","4,989,499.835"
3,2021 רבעון 2,"אג""ח קונצרני","32,024.025","4,618,577.703"
4,2021 רבעון 3,מניות,"89,079.454","5,436,705.671"
5,2021 רבעון 3,"אג""ח קונצרני","-8,400.337","4,652,915.640"
6,2021 רבעון 4,מניות,"873,591.632","5,849,256.429"
7,2021 רבעון 4,"אג""ח קונצרני","-73,279.754","4,239,169.274"
8,2022 רבעון 1,מניות,"1,098,591.830","7,045,177.277"
9,2022 רבעון 1,"אג""ח קונצרני","-115,848.031","4,157,998.344"


In [34]:
fossil_profits_by_type_and_q.groupby("quarter").sum()

,profit,total
quarter,,
2021 רבעון 1,"446,569.444","9,186,623.847"
2021 רבעון 2,"321,867.701","9,608,077.538"
2021 רבעון 3,"80,679.118","10,089,621.312"
2021 רבעון 4,"800,311.878","10,088,425.703"
2022 רבעון 1,"982,743.800","11,203,175.621"


## 2. for all (major) companies

In [73]:
all_holdings_cls_filtered_grouped = all_holdings_cls_filtered.copy()
all_holdings_cls_filtered_grouped["ParentCorpGroup"] = "כל החברות"
all_major_companies = ["כל החברות"]
quarters_for_comparison = [
    '2021 רבעון 2',
    '2021 רבעון 3'
]

compare_all = compare_fossil_holdings_over_quarters(
    all_holdings_cls_filtered_grouped,
    quarters_for_comparison,
    all_major_companies
)

total Israeli fossil holdings: 76144868.64174868
total fossil holdings with il_sec_num: 76144868.6417487
total fossil holdings without il_sec_num: 24143102.99176176
24143102.99176176
after adding Israeli and non-Israeli: 100287971.63351044


In [74]:
quarters = ' '.join(quarters_for_comparison)
compare_all.to_csv("analysis/all companies - fossil holdings comparison - " + quarters + ".csv", index=False)

## Look into non fossil holding types

In [53]:
all_holdings_cls_non_fossil_types = all_holdings_cls_filtered[
    all_holdings_cls_filtered["holding_type"].isin(get_non_fossil_holding_types())
]

all_holdings_cls_non_fossil_types['holding_type'].unique()

array(['מזומנים', 'תעודות התחייבות ממשלתיות',
       'לא סחיר - תעודות התחייבות ממשלתי', 'פקדונות מעל 3 חודשים'],
      dtype=object)

In [54]:
all_holdings_cls_non_fossil_types.groupby("holding_type").agg({
      "שווי": "sum",
    "שווי פוסילי": "sum"
})

,שווי,שווי פוסילי
holding_type,,
לא סחיר - תעודות התחייבות ממשלתי,"1,040,434,526.80",0.00
מזומנים,"738,522,129.92",0.00
פקדונות מעל 3 חודשים,"53,212,857.20",0.00
תעודות התחייבות ממשלתיות,"1,491,058,879.74","29,677.35"


In [55]:
# holdings that should be non fossil by holding type but are fossil
all_holdings_cls_non_fossil_types[
    all_holdings_cls_non_fossil_types["is_fossil"] == 1
]

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI,ParentCorpGroup,ReportPeriodDate
13173,ISRELE 4.25 14/8/28,NaN,NaN,Baa2,Moodys,דולר אמריקאי,0.04,0.03,"14,919.64",0.00,0.00,2049749,תעודות התחייבות ממשלתיות,NYSE,NaN,6.34,"4,000,000.00",111.87,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"מור קופות גמל בע""מ",I_514956465,0.0,nan,nan,27/05/2021,2021 רבעון 1,1.00,"14,919.64",IL0060002446,NaN,NaN,מור,2021-03-31
20932,ETP 5 1/4 04/15,NaN,NaN,NR,NaN,דולר אמריקאי,0.05,0.08,"2,366.65",0.01,0.00,1847126,תעודות התחייבות ממשלתיות,אחר,NaN,6.89,"576,000.00",115.25,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,08/07/2020,2020 רבעון 1,1.00,"2,366.65",US29278NAG88,NaN,5493008K8W3OKZE54J59,פסגות,2020-03-31
20936,PEMEX 0 03/11/2,NaN,NaN,NR,NaN,דולר אמריקאי,0.06,0.02,517.37,0.00,0.00,1847126,תעודות התחייבות ממשלתיות,אחר,NaN,0.19,"176,000.00",82.46,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,08/07/2020,2020 רבעון 1,1.00,517.37,US71654QCF72,NaN,NaN,פסגות,2020-03-31
102585,ETP 5 1/4 04/15,NaN,NaN,NR,NaN,דולר אמריקאי,0.05,0.04,"2,183.39",0.01,0.00,1945468,תעודות התחייבות ממשלתיות,אחר,NaN,7.60,"576,000.00",110.16,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,30/11/2020,2020 רבעון 3,1.00,"2,183.39",US29278NAG88,NaN,5493008K8W3OKZE54J59,פסגות,2020-09-30
102588,PEMEX 0 03/11/2,NaN,NaN,NR,NaN,דולר אמריקאי,0.06,0.05,600.23,0.00,0.00,1945468,תעודות התחייבות ממשלתיות,אחר,NaN,0.20,"176,000.00",99.11,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,30/11/2020,2020 רבעון 3,1.00,600.23,US71654QCF72,NaN,NaN,פסגות,2020-09-30
133682,ETP 5 1/4 04/15,NaN,NaN,NR,NaN,דולר אמריקאי,NaN,NaN,"2,204.06",0.01,0.00,1884615,תעודות התחייבות ממשלתיות,אחר,NaN,nan,"576,000.00",110.40,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,07/09/2020,2020 רבעון 2,1.00,"2,204.06",US29278NAG88,NaN,5493008K8W3OKZE54J59,פסגות,2020-06-30
133686,PEMEX 0 03/11/2,NaN,NaN,NR,NaN,דולר אמריקאי,NaN,NaN,592.68,0.00,0.00,1884615,תעודות התחייבות ממשלתיות,אחר,NaN,nan,"176,000.00",97.16,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,07/09/2020,2020 רבעון 2,1.00,592.68,US71654QCF72,NaN,NaN,פסגות,2020-06-30
146633,ETP 5 1/4 04/15,NaN,NaN,NR,NaN,דולר אמריקאי,NaN,0.00,"2,331.49",0.01,0.00,2062624,תעודות התחייבות ממשלתיות,אחר,NaN,0.00,"599,000.00",116.75,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,20/06/2021,2021 רבעון 1,1.00,"2,331.49",US29278NAG88,NaN,5493008K8W3OKZE54J59,פסגות,2021-03-31
146636,PEMEX 0 03/11/2,NaN,NaN,NR,NaN,דולר אמריקאי,NaN,0.00,859.55,0.00,0.00,2062624,תעודות התחייבות ממשלתיות,אחר,NaN,0.00,"254,000.00",101.50,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,20/06/2021,2021 רבעון 1,1.00,859.55,US71654QCF72,NaN,NaN,פסגות,2021-03-31
403299,ETP 5 1/4 04/15,NaN,NaN,NR,NaN,דולר אמריקאי,NaN,NaN,"2,273.40",0.01,0.00,2018568,תעודות התחייבות ממשלתיות,אחר,NaN,0.00,"599,000.00",118.05,0.00,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,05/04/2021,2020 רבעון 4,1.00,"2,273.40",US29278NAG88,NaN,5493008K8W3OKZE54J59,פסגות,2020-12-31


## Export holdings with missing classification that should be classified

In [9]:
missing_cls = all_holdings_cls[
    (all_holdings_cls["holding_type"].isin(['מניות', 'אג"ח קונצרני'])) &
    (all_holdings_cls["is_fossil"].isnull())
]

cols = ['שם המנפיק/שם נייר ערך' ,'מספר ני"ע' ,'מספר מנפיק', 'ISIN', 'מספר תאגיד', 'LEI']
missing_cls_deduped = missing_cls[cols].drop_duplicates()
print(missing_cls_deduped.shape[0])
missing_cls_deduped

13


,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,ISIN,מספר תאגיד,LEI
28731,CHKP US Equity- צ'ק פוינט,IL0010824113 EQUITY,NaN,NaN,520042821,NaN
38221,PLURISTEM Therapeutics Inc SYS- PLURISTEM THER...,US72940P1066-71013122,NaN,NaN,NaN,NaN
52733,PEMEX 5.95 28/01/31,USP71654QDE98,NaN,NaN,NaN,NaN
56742,GLAXOSMITHKLINE PLC- GLAXOSMITHKLINE PLC,US37733W1053-110637592,NaN,NaN,NaN,NaN
94603,Tencent holdings- Tencent holdings ltd,KY6875721220-60175411,NaN,NaN,NaN,NaN
114477,איי.אפ.אפ- אינטרנשיונל פליוורס אנד פראגרנסס אינק,US4595061015_1155019,1760,NaN,NaN,NaN
122042,Capital One- capital one,US14040H1059-60004140,NaN,NaN,NaN,NaN
141181,P-ADR,70422225,NaN,NaN,NaN,NaN
181606,LVS VEGAS SANDS CORP- LAS VEGAS SANDS CORP,71032965 - US5178341070,NaN,NaN,NaN,NaN
292935,CYBR 0 11/15/2024,US23248VAA35 C,NaN,NaN,NaN,NaN


In [15]:
missing_cls_deduped.to_csv(response_path+"missing_cls.csv", index=False)

# Simulation - change fossil classification for selected holdings, analyze results

In [17]:
# fix data in all_holdings_cls - wrong data for סופרגז
# all_holdings_cls[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז']

In [49]:
all_holdings_cls.loc[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז',
                    'מספר ני"ע'] = '1166917'

all_holdings_cls.loc[all_holdings_cls["שם המנפיק/שם נייר ערך"] == 'סופרגז',
                     'ISIN'] = 'IL0011669178'

In [50]:
all_holdings_cls["clean_name"] = all_holdings_cls["שם המנפיק/שם נייר ערך"].apply(clean_company)

In [68]:
selected_holdings_names = ["כיל", "ICL", 'איי סי אל', 'איי.סי.אל', 'אייסיאל', 'ISRAEL CHEMICAL', 'כימיקלים לישראל', 'כי"ל',
                           "רימון",
                           "סופרגז", "סופר גז",
                           "OPC", 'אופיסי', 'או. פי. סי', 'או פי סי',
                           'חברה לישראל', 'החברה לישראל', 'חברהלישראל','חברה ישראל']
name_mask = all_holdings_cls["clean_name"].str.upper().str.startswith(tuple(selected_holdings_names))
all_holdings_cls.loc[name_mask].groupby(['מספר מנפיק'], dropna=False).agg(
    name=pd.NamedAgg(column="clean_name", aggfunc="first")
)

,name
מספר מנפיק,
1395,רימון מניה לא סחירה
1486,סופרגז
1682,אופיסי
1804,סופרגז
1928,רימון
281,אייסיאל
28327,OPC POWER VENTURES LP
576,חברה לישראל
NaN,סופרגז


In [69]:
def holdings_col_mask_by_another_mask(holdings, col_name, mask):
    selected_col_values = holdings.loc[mask, col_name].dropna().unique()
    col_mask = holdings[col_name].isin(selected_col_values)
    print("matching holdings by {}: {}".format(col_name, col_mask.sum()))
    return col_mask

In [70]:
# by issuer num
issuer_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "מספר מנפיק", name_mask)
ISIN_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "ISIN", name_mask)
LEI_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "LEI", name_mask)
il_corp_mask = holdings_col_mask_by_another_mask(all_holdings_cls, "מספר תאגיד", name_mask)
il_sec_num_mask = holdings_col_mask_by_another_mask(all_holdings_cls, 'מספר ני"ע', name_mask)

matching holdings by מספר מנפיק: 3216
matching holdings by ISIN: 2784
matching holdings by LEI: 0
matching holdings by מספר תאגיד: 3317
matching holdings by מספר ני"ע: 3104


In [71]:
all_ids_mask = name_mask | issuer_mask | ISIN_mask | LEI_mask | il_corp_mask | il_sec_num_mask
no_name_match = all_holdings_cls[all_ids_mask & (~name_mask)]
print("selected holdings with no name match: {}".format(no_name_match.shape[0]))

selected holdings with no name match: 234


In [74]:
no_name_match.sort_values("is_fossil", ascending=False)

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI,clean_name
303258,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.04,"3,604.36",0.01,0.00,1824549,"אג""ח קונצרני",TASE,NaN,2.39,"3,475,752.20",103.70,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,27/05/2020,2020 רבעון 1,1.00,"3,604.36",IL0057602364,520028010,NaN,חלישראל אג
516755,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.0114,"2,119.54",0.01,0.00,2150933,"אג""ח קונצרני",TASE,NaN,1.68,"1,974,053.22",107.37,0.00,0.00,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0,nan,nan,02/12/2021,2021 רבעון 3,1.00,"2,119.54",IL0057602364,520028010,NaN,חלישראל אג
260646,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.04,0.01,"5,154.03",0.02,0.00,2014144,"אג""ח קונצרני",TASE,NaN,2.03,"4,815,945.98",107.02,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,24/03/2021,2020 רבעון 4,1.00,"5,154.03",IL0057602364,520028010,NaN,חלישראל אג
149389,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.03,"3,218.08",0.01,0.00,1879029,"אג""ח קונצרני",TASE,NaN,2.47,"3,089,557.51",104.16,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,30/08/2020,2020 רבעון 2,1.00,"3,218.08",IL0057602364,520028010,NaN,חלישראל אג
203528,ISRAE 4.5 12/24,245217,281,ilBBB-,S&P מעלות,דולר אמריקאי,0.04,0.02,"3,658.36",0.00,0.00,1948374,"אג""ח קונצרני",TASE,NaN,3.82,"960,000.00",110.75,0.00,0.00,NaN,Materials,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"מיטב דש גמל ופנסיה בע""מ",I_512065202,0.0,nan,nan,06/12/2020,2020 רבעון 3,1.00,"3,658.36",None,520027830,NaN,ISRAE
306998,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.02,"4,758.65",0.02,0.00,1943203,"אג""ח קונצרני",TASE,NaN,2.24,"4,415,557.51",107.77,0.00,0.01,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,26/11/2020,2020 רבעון 3,1.00,"4,758.65",IL0057602364,520028010,NaN,חלישראל אג
470334,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.02,"2,600.16",0.01,0.00,2051177,"אג""ח קונצרני",TASE,NaN,1.79,"2,429,603.98",107.02,0.00,0.00,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,31/05/2021,2021 רבעון 1,1.00,"2,600.16",IL0057602364,520028010,NaN,חלישראל אג
468511,ISRAE 4.5 12/24,None,281,BBB-,S&P,דולר אמריקאי,0.04,0.01,"3,484.24",0.00,0.00,2016501,"אג""ח קונצרני",אחר,NaN,3.65,"960,000.00",112.89,0.00,0.00,בלומברג,Materials,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"מיטב דש גמל ופנסיה בע""מ",I_512065202,0.0,nan,nan,31/03/2021,2020 רבעון 4,1.00,"3,484.24",IL0028102734,520027830,NaN,ISRAE
456335,ח.לישראל אג10 3.85%,5760236,576,ilA,S&P מעלות,שקל חדש,0.05,0.01,"2,115.20",0.01,0.00,2104058,"אג""ח קונצרני",TASE,NaN,1.93,"1,974,053.22",107.15,0.00,0.00,NaN,השקעה ואחזקות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת תגמולים של עובדי התעשיה האוירית לישראל בע""מ",I_570014928,0.0,nan,nan,02/09/2021,2021 רבעון 2,1.00,"2,115.20",IL0057602364,520028010,NaN,חלישראל אג
437836,ISHARES S&P GBL CLEAN ENERGY,None,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,16.80,0.00,0.00,1949813,קרנות סל,NYSE,NaN,nan,264.00,"1,849.00",0.00,0.00,NaN,מניות,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"פסגות קופות גמל ופנסיה בע""מ",I_513765347,0.0,nan,nan,07/12/2020,

In [73]:
all_holdings_cls.loc[all_ids_mask, "is_fossil"].value_counts(dropna=False)

1.00    3193
nan      304
0.00      32
Name: is_fossil, dtype: int64

In [103]:
# change classification
all_holdings_cls.loc[all_ids_mask, "is_fossil"] = 0
all_holdings_cls["שווי פוסילי"] = all_holdings_cls["שווי"] * all_holdings_cls["is_fossil"]

## Misc queries

In [9]:
response_path = "data/downloaded reports/company reports 2022Q1/"
all_holdings_cls = pd.read_csv(response_path+"all_holdings_cls_all_ids.csv", dtype=holdings_dtypes())
# all_holdings_cls = load_mappings_and_add_ids_to_holdings(all_holdings_cls)
current_q = '2022 רבעון 1'

/Users/urimarom/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,7,13,20,22,23,24,25,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## holding type distribution (asset allocation)

In [18]:
latest_q_sums_by_htype = all_holdings_cls[
    (all_holdings_cls["ReportPeriodDesc"] == current_q)
].groupby("holding_type")[['שווי']].agg('sum').reset_index()

latest_q_sums_by_htype["pct"] = 100.00 * latest_q_sums_by_htype["שווי"] / latest_q_sums_by_htype["שווי"].sum()

print(latest_q_sums_by_htype["שווי"].sum())
latest_q_sums_by_htype.sort_values('שווי', ascending=False).to_clipboard(index=False)
latest_q_sums_by_htype.sort_values('שווי', ascending=False)

1778911368.393303


,holding_type,שווי,pct
19,מניות,"356,174,475.806",20.022
26,תעודות התחייבות ממשלתיות,"266,786,114.565",14.997
15,לא סחיר - תעודות התחייבות ממשלתי,"194,035,244.409",10.908
25,קרנות סל,"176,950,258.872",9.947
0,"אג""ח קונצרני","176,710,621.725",9.934
18,מזומנים,"173,550,797.754",9.756
14,לא סחיר - קרנות השקעה,"148,331,815.756",8.338
2,הלוואות,"122,906,255.489",6.909
24,קרנות נאמנות,"55,313,023.582",3.109
13,לא סחיר - מניות,"29,080,001.513",1.635


## Non-major companies

In [19]:
haifa = all_holdings_cls.loc[
    (all_holdings_cls["ParentCorpName"].str.contains("חיפה"))
]

In [20]:
get_summary(haifa, 'ReportPeriodDesc', 'ParentCorpName')

,ReportPeriodDesc,ParentCorpName,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים,שווי בסוגי החזקות לא פוסיליים,שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים
0,2020 רבעון 1,"קופת""ג של עובדי עירית חיפה","240,715.370","13,613.360",0.057,"108,454.540","13,601.530",0.125,"87,596.720",0.069
1,2020 רבעון 2,"קופת""ג של עובדי עירית חיפה","250,150.810","11,857.890",0.047,"108,377.360","11,844.990",0.109,"85,507.360",0.061
2,2020 רבעון 3,"קופת""ג של עובדי עירית חיפה","255,221.800","8,596.730",0.034,"100,511.100","8,594.490",0.086,"90,400.320",0.045
3,2020 רבעון 4,"קופת""ג של עובדי עירית חיפה","260,795.440","8,213.870",0.031,"107,191.310","8,211.860",0.077,"81,635.060",0.043
4,2021 רבעון 1,"קופת""ג של עובדי עירית חיפה","262,667.780","8,806.960",0.034,"108,852.410","8,806.190",0.081,"77,062.770",0.047
5,2021 רבעון 2,"קופת""ג של עובדי עירית חיפה","263,148.260","9,159.620",0.035,"112,306.530","9,159.520",0.082,"73,023.550",0.049
6,2021 רבעון 3,"קופת""ג של עובדי עירית חיפה","258,155.800","6,909.880",0.027,"95,726.140","6,909.880",0.072,"71,384.130",0.041
7,2021 רבעון 4,"קופת""ג של עובדי עירית חיפה","263,032.120","9,535.330",0.036,"96,427.180","9,535.330",0.099,"80,815.320",0.054
8,2022 רבעון 1,"קופת""ג של עובדי עירית חיפה","253,564.810","9,502.960",0.037,"92,636.380","9,502.960",0.103,"75,584.930",0.056


In [21]:
haifa[haifa["is_fossil"] == 1].sort_values("שם המנפיק/שם נייר ערך")

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI
323136,DELEK(DK US),NaN,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,114.000,0.002,0.001,1816306,מניות,NYSE,NaN,nan,"2,029.000","1,576.000",0.000,0.000,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,13/05/2020,2020 רבעון 1,1.000,114.000,US2466471016,520032681,NaN
306831,DEVTAM 4.435 12/30/20,NaN,NaN,Baa3,MOODYS,דולר אמריקאי,0.044,0.029,"2,782.030",0.049,0.011,1859023,"אג""ח קונצרני",TASE,NaN,0.480,"800,000.000",100.330,0.000,0.002,בלומברג,חיפושי נפט וגז,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,28/07/2020,2020 רבעון 2,1.000,"2,782.030",IL0011321663,514914001,NaN
222558,DEVTAM 4.435 12/30/20,NaN,NaN,Baa3,MOODYS,דולר אמריקאי,0.044,0.035,698.170,0.013,0.003,1923377,"אג""ח קונצרני",TASE,NaN,0.240,"200,000.000",101.450,0.000,0.001,בלומברג,חיפושי נפט וגז,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,01/11/2020,2020 רבעון 3,1.000,698.170,IL0011321663,514914001,NaN
322992,DEVTAM 4.435 12/30/20,NaN,NaN,Baa3,MOODYS,דולר אמריקאי,0.044,0.044,"2,879.340",0.050,0.012,1816306,"אג""ח קונצרני",TASE,NaN,0.710,"800,000.000",100.960,0.000,0.002,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,13/05/2020,2020 רבעון 1,1.000,"2,879.340",IL0011321663,514914001,NaN
323116,DIAMONDBACK ENERGY INC,NaN,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,28.770,0.001,0.000,1816306,מניות,אחר,NaN,nan,308.000,"2,620.000",0.000,0.000,בלומברג,Other,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,13/05/2020,2020 רבעון 1,1.000,28.770,US25278X1090,NaN,549300R22LSX6OHWEN64
170226,E.ON SE,NaN,NaN,NaN,NaN,אירו,NaN,NaN,399.570,0.007,0.002,1980276,מניות,אחר,NaN,nan,"11,177.000",906.400,0.000,0.000,בלומברג,Other,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,26/01/2021,2020 רבעון 4,1.000,399.570,DE000ENAG999,NaN,Q9MAIUP40P25UFBFG033
306948,E.ON SE,NaN,NaN,NaN,NaN,אירו,NaN,NaN,434.630,0.009,0.002,1859023,מניות,אחר,NaN,nan,"11,177.000","1,001.500",0.000,0.000,בלומברג,Other,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,28/07/2020,2020 רבעון 2,1.000,434.630,DE000ENAG999,NaN,Q9MAIUP40P25UFBFG033
343310,E.ON SE,NaN,NaN,NaN,NaN,אירו,NaN,NaN,422.430,0.007,0.002,2084579,מניות,אחר,NaN,nan,"11,177.000",975.400,0.000,0.000,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,28/07/2021,2021 רבעון 2,1.000,422.430,DE000ENAG999,NaN,Q9MAIUP40P25UFBFG033
323117,E.ON SE,NaN,NaN,NaN,NaN,אירו,NaN,NaN,411.570,0.008,0.002,1816306,מניות,אחר,NaN,nan,"11,177.000",944.100,0.000,0.000,בלומברג,Other,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,13/05/2020,2020 רבעון 1,1.000,411.570,DE000ENAG999,NaN,Q9MAIUP40P25UFBFG033
151707,E.ON SE,NaN,NaN,NaN,NaN,אירו,NaN,NaN,434.000,0.007,0.002,2032597,מניות,אחר,NaN,nan,"11,177.000",992.400,0.000,0.000,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,גמל,"קופת""ג של עובדי עירית חיפה",I_570005959,0.0,nan,nan,28/04/2021,2021 רבעון 1,1.000,434.000,DE000ENAG999,NaN,Q9MAIUP40P25UFBFG033


In [22]:
hachshara = all_holdings_cls.loc[
    (all_holdings_cls["ParentCorpName"].str.contains("הכשר"))
]
get_summary(hachshara, 'ReportPeriodDesc', 'ParentCorpName')

,ReportPeriodDesc,ParentCorpName,שווי,שווי פוסילי,שיעור פוסילי מסך הנכסים,שווי במניות ואגח קונצרני סחירים,שווי פוסילי במניות ואגח קונצרני סחירים,שיעור פוסילי במניות ואגח קונצרני סחירים,שווי בסוגי החזקות לא פוסיליים,שיעור פוסילי מתוך מניות ואגח סחירים + סוגי החזקות לא פוסיליים
0,2020 רבעון 1,"הכשרה חברה לביטוח בע""מ","14,286,995.881","466,987.813",0.033,"4,632,085.754","466,987.813",0.101,"6,525,615.549",0.042
1,2020 רבעון 2,"הכשרה חברה לביטוח בע""מ","15,662,871.060","472,662.323",0.030,"4,857,582.499","472,662.323",0.097,"7,054,616.386",0.040
2,2020 רבעון 3,"הכשרה חברה לביטוח בע""מ","16,923,932.260","463,914.967",0.027,"5,199,067.218","463,914.967",0.089,"7,914,866.098",0.035
3,2021 רבעון 1,"הכשרה חברה לביטוח בע""מ","21,250,381.045","576,146.786",0.027,"6,759,953.345","576,034.757",0.085,"9,082,732.353",0.036
4,2021 רבעון 4,"הכשרה חברה לביטוח בע""מ","24,761,410.596","740,718.166",0.030,"7,448,277.008","740,695.176",0.099,"10,645,423.368",0.041
5,2022 רבעון 1,"הכשרה חברה לביטוח בע""מ","23,829,392.108","799,193.430",0.034,"7,227,988.023","799,156.576",0.111,"10,041,397.621",0.046


## looking into specific holdings
קרנות סל - מגדל 125

In [549]:
all_holdings_2021q2[
    (all_holdings_2021q2["holding_type"] == 'קרנות סל') &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains("125")) &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains("מגדל"))
].groupby('מספר ני"ע')["שווי"].sum()

מספר ני"ע
1150283   89,073.06
1159714   37,317.87
1174796       17.59
Name: שווי, dtype: float64

In [569]:
# החזקות של חברות בקרן עוקבת תא 125 של מגדל
cols = ["SystemName", "ParentCorpName","שווי", 'שעור מנכסי אפיק ההשקעה', "שעור מערך נקוב מונפק"]
all_holdings_2021q2[
    (all_holdings_2021q2['מספר ני"ע'] == 1150283)
][cols]

,SystemName,ParentCorpName,שווי,שעור מנכסי אפיק ההשקעה,שעור מערך נקוב מונפק
13831,גמל,"הלמן - אלדובי קופות גמל ופנסיה בע""מ",231.34,0.00,0.00
18897,פנסיה,"מיטב דש גמל ופנסיה בע""מ","16,149.28",0.01,0.01
29346,ביטוח,"איילון חברה לביטוח בע""מ",560.67,0.00,0.00
40372,גמל,"פסגות קופות גמל ופנסיה בע""מ","16,325.29",0.01,0.02
74680,גמל,"הפניקס פנסיה וגמל בע""מ","1,050.54",0.00,0.00
122078,גמל,"מיטב דש גמל ופנסיה בע""מ","25,739.15",0.01,0.02
160766,גמל,"החברה לניהול קרן ההשתלמות להנדסאים וטכנאים בע""מ","1,266.51",0.00,0.00
248891,פנסיה,"פסגות קופות גמל ופנסיה בע""מ","9,443.80",0.03,0.01
279942,גמל,"אינפיניטי ניהול השתלמות וגמל בע""מ","3,342.95",0.01,0.00
398997,גמל,"החברה לניהול קרן השתלמות למשפטנים בע""מ",134.76,0.00,0.00


In [575]:
cols = ["שם המנפיק/שם נייר ערך", 'מספר ני"ע', "שווי", "שעור מנכסי אפיק ההשקעה", "שעור מערך נקוב מונפק", "SystemName"]

all_holdings_2021q2[
    (all_holdings_2021q2['holding_type'] == 'קרנות סל') &
    (all_holdings_2021q2["ParentCorpName"].str.contains('מגדל')) &
    (~all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains('כש')) &
    (all_holdings_2021q2["שם המנפיק/שם נייר ערך"].str.contains('125'))
][cols]

,שם המנפיק/שם נייר ערך,"מספר ני""ע",שווי,שעור מנכסי אפיק ההשקעה,שעור מערך נקוב מונפק,SystemName
88639,הראל סל תא 125,1148899,"77,115.37",0.01,0.09,ביטוח
88644,פסגות ETF תל אביב 125,1148808,"155,533.51",0.01,0.12,ביטוח
88648,קסם תא125,1146356,"140,897.26",0.01,0.07,ביטוח
88650,תכלית תא 125,1143718,"47,261.98",0.00,0.02,ביטוח
281587,"הראל סל תא 125- הראל קרנות נאמנות בע""מ",1148899,"14,195.15",0.01,0.02,גמל
281591,"*MTF סל )4A( ת""א125- אקלים נקי- מגדל קרנות נאמ...",1174796,17.59,0,0,גמל
281596,"תכלית סל תא 125- מיטב תכלית קרנות נאמנות בע""מ",1143718,"9,595.19",0.00,0.00,גמל
281599,"פסגות ETF תא 125- פסגות קרנות מדדים בע""מ",1148808,"27,112.11",0.01,0.02,גמל
281606,"קסם ETF תא 125- קסם קרנות נאמנות בע""מ",1146356,"24,544.03",0.01,0.01,גמל
455627,הראל סל תא 125,1148899,"56,424.02",0.01,0.06,פנסיה


## Looking into ETFs

In [557]:
etf_holdings = all_holdings_2021q2[
    (all_holdings_2021q2["holding_type"] == 'קרנות סל')
]

etf_holdings.groupby("זירת מסחר")["שווי"].sum()

זירת מסחר
AMEX              118.70
ASX           483,289.25
CAC           530,710.30
DAX           121,456.62
EURONEXT    4,150,375.49
FWB         2,293,354.79
HKSE        1,230,909.60
ISE            28,392.81
JPX           187,602.86
LSE        15,767,218.60
NASDAQ      9,568,288.10
NYSE       39,434,670.96
SIX         1,638,552.83
TASE       30,436,824.11
TSE         4,318,613.35
TSX            26,454.18
אחר        13,829,003.07
Name: שווי, dtype: float64

In [562]:
etf_holdings_tase = etf_holdings[etf_holdings["זירת מסחר"] == "TASE"]
etf_holdings_tase.head()

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc
8585,MTF סל S&P 500 מנוטר,1150572,511303661,NaN,NaN,שקל חדש,NaN,NaN,"1,372.16",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,23847,5754,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8586,MTF סל indx USA clou,1158328,511303661,NaN,NaN,שקל חדש,NaN,NaN,209.26,0.00,0.00,2098344,קרנות סל,TASE,NaN,NaN,3935,5318,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8587,הראל סל NDX 100 מנוט,1149103,511776783,NaN,NaN,שקל חדש,NaN,NaN,"1,439.72",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,10328,13940,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8588,הראל סל SP500 מנוטרל,1149137,511776783,NaN,NaN,שקל חדש,NaN,NaN,"1,727.06",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,38793,4452,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2
8589,מו.NDX100ממ,1165844,514884485,NaN,NaN,שקל חדש,NaN,NaN,"1,466.68",0.02,0.00,2098344,קרנות סל,TASE,NaN,NaN,19863,7384,0,0.00,NaN,מניות,NaN,NaN,NaN,NaN,NaN,NaN,NaN,גמל,"שיבולת - חברה לניהול קופות גמל בע""מ",520030693,0,nan,nan,22/08/2021,2021 רבעון 2


### Latest quarter holdings per company

In [13]:
all_holdings_cls_latest_q = all_holdings_cls[
    all_holdings_cls["ReportPeriodDesc"] == '2021 רבעון 3'
]

pd.DataFrame(all_holdings_cls_latest_q.groupby(
    ['ParentCorpName'])['שווי'].agg('sum')
            ).sort_values('שווי', ascending=False)

,שווי
ParentCorpName,
"מנורה מבטחים פנסיה וגמל בע""מ","207,714,405.668"
"אלטשולר שחם גמל ופנסיה בע""מ","194,508,347.667"
"מגדל חברה לביטוח בע""מ","138,241,279.585"
"הראל פנסיה וגמל בע""מ","137,150,641.307"
"כלל פנסיה וגמל בע""מ","130,887,924.452"
"מגדל מקפת קרנות פנסיה וקופות גמל בע""מ","117,578,511.132"
"כלל חברה לביטוח בע""מ","88,412,287.919"
"הפניקס פנסיה וגמל בע""מ","86,005,354.910"
"הפניקס חברה לביטוח בע""מ","74,882,609.120"


In [13]:
all_holdings_cls_filtered.groupby("holding_type")["שווי"].sum().sort_values()

holding_type
השקעה בחברות מוחזקות                      -8,247.000
עלות מתואמת אג"ח קונצרני סחיר            122,928.343
תעודות חוב מסחריות                       258,039.961
לא סחיר - אופציות                        912,090.563
כתבי אופציה                            1,416,505.348
אופציות                                2,239,004.711
לא סחיר - כתבי אופציה                  2,572,139.501
לא סחיר - תעודות חוב מסחריות           2,936,403.382
עלות מתואמת מסגרות אשראי ללווים        3,001,613.612
חוזים עתידיים                          4,377,029.694
עלות מתואמת אג"ח קונצרני ל.סחיר        6,966,352.249
השקעות אחרות                           8,563,186.503
לא סחיר - חוזים עתידיים               25,734,366.181
מוצרים מובנים                         29,175,656.208
לא סחיר - מוצרים מובנים               38,420,566.434
פקדונות מעל 3 חודשים                  61,547,127.772
זכויות מקרקעין                       148,954,505.348
לא סחיר - מניות                      157,320,067.240
לא סחיר - אג"ח קונצרני           

In [15]:
all_holdings_cls_filtered[
    all_holdings_cls_filtered["holding_type"].str.contains("קרנות השקעה")
    ].head()

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI,ParentCorpGroup,ReportPeriodDate
2246,Greenfield Partners Fund I,33445,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,"36,212.254",0.006,0.001,1828503,לא סחיר - קרנות השקעה,NaN,04/03/20,nan,"10,157,714.870",100.000,nan,0.000,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"הפניקס חברה לביטוח בע""מ",I_520023185,0.0,nan,nan,03/06/2020,2020 רבעון 1,nan,nan,NaN,NaN,NaN,הפניקס,2020-03-31
2247,ICV II,33408,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,"17,320.986",0.003,0.000,1828503,לא סחיר - קרנות השקעה,NaN,07/08/19,nan,"3,073,200.000",158.097,nan,0.000,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"הפניקס חברה לביטוח בע""מ",I_520023185,0.0,nan,nan,03/06/2020,2020 רבעון 1,nan,nan,NaN,NaN,NaN,הפניקס,2020-03-31
2248,Glilot Capital Partners II,33116,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,"23,292.112",0.004,0.000,1828503,לא סחיר - קרנות השקעה,NaN,04/05/15,nan,"4,765,168.950",137.111,nan,0.085,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"הפניקס חברה לביטוח בע""מ",I_520023185,0.0,nan,nan,03/06/2020,2020 רבעון 1,nan,nan,NaN,NaN,NaN,הפניקס,2020-03-31
2249,Glilot Capital Partners III,33390,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,"2,502.775",0.000,0.000,1828503,לא סחיר - קרנות השקעה,NaN,15/01/19,nan,"1,020,289.510",68.808,nan,0.000,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"הפניקס חברה לביטוח בע""מ",I_520023185,0.0,nan,nan,03/06/2020,2020 רבעון 1,nan,nan,NaN,NaN,NaN,הפניקס,2020-03-31
2250,Wanaka Capital Partners Mid–Tech Opportunity Fun,32081,NaN,NaN,NaN,דולר אמריקאי,NaN,NaN,252.251,0.000,0.000,1828503,לא סחיר - קרנות השקעה,NaN,31/03/08,nan,"1,931,213.200",3.664,nan,0.034,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,nan,ביטוח,"הפניקס חברה לביטוח בע""מ",I_520023185,0.0,nan,nan,03/06/2020,2020 רבעון 1,nan,nan,NaN,NaN,NaN,הפניקס,2020-03-31
